![Neo4j version](https://img.shields.io/badge/Neo4j->=4.4.9-brightgreen)
![GDS version](https://img.shields.io/badge/GDS->=2.1-brightgreen)
![GDS Python Client version](https://img.shields.io/badge/GDS_Python_Client->=1.4-brightgreen)

In [1]:
%%capture
%pip install graphdatascience python-dotenv

## Notebook Setup

In [2]:
from graphdatascience import GraphDataScience
import pandas as pd
from dotenv import load_dotenv
import os

### Connect to Neo4j Graph Data Science
Create a `db-credentials.env` from template the template and fill in credentials as appropriate.

In [3]:
load_dotenv('db-credentials.env', override=True)

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

### Clean Up from Last Time

In [4]:
# remove old rels
_ = gds.run_cypher('MATCH (:RecentNews)-[r:USERS_ALSO_LIKED]->() DELETE r')

In [5]:
# remove properties
_ = gds.run_cypher('MATCH (n:News) REMOVE n.embedding')

In [6]:
# drop all the projected graphs
g_names = gds.graph.list().graphName.tolist()

for g_name in g_names:
    g = gds.graph.get(g_name)
    gds.graph.drop(g)

### Stage Graph

In [7]:
# create recent news label
gds.run_cypher('''
    MATCH(n:News)<-[:CLICKED|HISTORICALLY_CLICKED]-() WHERE n.approxTime IS NOT NULL
    SET n:RecentNews
    RETURN count(DISTINCT n)
    ''')

,count(DISTINCT n)
0,21858
